In [1]:
import pandas as pd
import dask as dd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import statsmodels.formula.api as smf
import statistics

from sklearn.linear_model import LogisticRegression

import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod import families
from statsmodels.stats.outliers_influence import variance_inflation_factor

from scipy import stats

/usr/local/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
sns.set()

## Filter out non automotive-related alliances

In [3]:
df = pd.read_csv('../data.csv')
df = df[df['psic'].notna()] #drop 45 NaN participant SIC, fix the issue by not solving it
autoDF = df[df['psicp'].str.contains('3711')] #Create df who have at least one participant in the automotive industry with SIC: 3711
autoDF = autoDF.drop('Unnamed: 0', axis=1)
autoDF = autoDF.reset_index(drop=True)

#Export
autoDF.to_csv('./data/autodf.csv', index=False) #create csv sample

/var/folders/5h/tc1b2dhx483_cx6pj09_pyc80000gn/T/ipykernel_8367/2145547248.py:1: DtypeWarning: Columns (3,4,7,8,11,12,14,15,16,19,45,46,49,56,69,83,95,98,99,100,104,108,109,119,120,121,122,123,124,125,127,128,129,141,143,144,145,146,147,148,155,188,217,218,219,220,221,222,223,224,226,229,232,233,234,235,236,237,238,239,240,243,244,245,246,248,249,250,252) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data.csv')


## Data selection

### Filter variables

In [5]:
variables = ["id","activity", "activityc", "da", "jvinc", "jvindustry", "jvstatus", "p", "pbl", 
		   "pbuss", "sicp", "sic", "SICPDESC".lower(), "psic", "psicp", "nump", "jvf", "jvtype", 
		   "rndf", "pemp", "PBUSSOURCE".lower(), "HITECHC".lower(), "crlic", "crtech", "TECHNIQUEC".lower(),
		   "TECHTR".lower(), "PPUBC".lower(), "SNATION_PARTAL".lower(), 'CR_BOR_PART'.lower(), 'mfgf', 'natc',
		   "jvemp", "saf"]
filteredDF = autoDF[variables]

### Select timeframe

In [6]:
filteredDF['da'] = pd.to_datetime(filteredDF['da']) 
timedDF = filteredDF[(filteredDF['da'] >= '2002-01-01') & (filteredDF['da'] <= '2012-12-31')]
timedDF.set_index('da', inplace=True, drop=True)

/var/folders/5h/tc1b2dhx483_cx6pj09_pyc80000gn/T/ipykernel_8367/2738038061.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filteredDF['da'] = pd.to_datetime(filteredDF['da'])


## Feature engineering

### Split PSICPs to seperate columns

In [7]:
pattern = r'\\n' #Regex pattern to split name

for n in range(1, (int(timedDF["nump"].max()) + 1)): 
	timedDF['p' + str(n) + "name"] =  timedDF['p'].str.split(pattern, expand=True)[n-1] #Create seperate columns for each possible participant

	timedDF["p" + str(n) + "sicp"] = timedDF['psicp'].str.split(pattern, expand=True)[n-1] #Create seperate placeholder columns to split participant SIC codes

/var/folders/5h/tc1b2dhx483_cx6pj09_pyc80000gn/T/ipykernel_8367/1130787018.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  timedDF['p' + str(n) + "name"] =  timedDF['p'].str.split(pattern, expand=True)[n-1] #Create seperate columns for each possible participant
/var/folders/5h/tc1b2dhx483_cx6pj09_pyc80000gn/T/ipykernel_8367/1130787018.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  timedDF["p" + str(n) + "sicp"] = timedDF['psicp'].str.split(pattern, expand=True)[n-1] #Create seperate placeholder col

### Define incumbent / new-entrant alliance

In [8]:
cols = timedDF.filter(regex='p\d+sicp', axis=1).columns #Get columns of individual participant SICs

timedDF['pdynamic'] = (timedDF[cols]=='3711').sum(axis=1) > 1 #Flag True for alliances consisting of more than one automotive incumbent
timedDF['pdynamic'] = timedDF['pdynamic'].map({True: "incumbent", False: "new_entrant"}) #Turn tha above from boolean to string

/var/folders/5h/tc1b2dhx483_cx6pj09_pyc80000gn/T/ipykernel_8367/3940381924.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  timedDF['pdynamic'] = (timedDF[cols]=='3711').sum(axis=1) > 1 #Flag True for alliances consisting of more than one automotive incumbent
/var/folders/5h/tc1b2dhx483_cx6pj09_pyc80000gn/T/ipykernel_8367/3940381924.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  timedDF['pdynamic'] = timedDF['pdynamic'].map({True: "incumbent", False: "new_entrant"}) #Turn tha above from boolean to s

In [9]:
#Export
timedDF.to_csv('./data/JV_data_2002-2012.csv')

### Generate discontintuiy time distance variations

In [10]:
JV_data_dist = timedDF.copy() ##Make copy

disc_day = pd.to_datetime("02/06/2007", format="%d/%m/%Y")

#### Integer distance

In [11]:
JV_data_dist['ddist_int'] = ((disc_day - JV_data_dist.index).days) * -1 #time -1 to reverse days count, negative should be before 2007

#### Natural num distance (absolute)

In [12]:
JV_data_dist['ddist_abs'] = abs((disc_day - JV_data_dist.index).days)

#### Binary distance

In [13]:
#Let 0 describe alliances before discontinuity day
#Let 1 describe alliances after discontinuity day

JV_data_dist['ddist_bin'] = 0 #Create placeholder columnc
JV_data_dist.loc[JV_data_dist.index > disc_day, 'ddist_bin'] = 1

### Year distance

In [14]:
JV_data_dist['ddist_year'] = (disc_day.year - JV_data_dist.index.year) * -1

### Split participant employee count to seperate variables

In [15]:
pattern = r'\\n'
pattern2 = r'p\d+emp'

for n in range(1, (int(JV_data_dist["nump"].max()) + 1)): 
	JV_data_dist["p" + str(n) + "emp"] =  pd.to_numeric(JV_data_dist['pemp'].str.split(pattern, expand=True)[n-1]) #Create seperate columns for each possible participant
	JV_data_dist["p" + str(n) + "emp"].replace(to_replace='', value=None, inplace=True) #Replace unknown employee count with na

foo = JV_data_dist.filter(regex=pattern2, axis=1).notna() #Get columns of individual participant employee count

JV_data_dist['known_emp'] = foo.any(axis=1) #Binary if employee count is known for >= participant
JV_data_dist['known_allemp'] = JV_data_dist['nump'] == foo.sum(axis=1) #Binary if employee count is known for ALL participants

#Get log of AVG employess per participant
fooReal = JV_data_dist.filter(regex=pattern2, axis=1) 
JV_data_dist['avg_emp_pp_log'] = np.log(fooReal.mean(axis=1, skipna=True))
JV_data_dist['avg_emp_pp'] = fooReal.mean(axis=1, skipna=True)

#Get mininmum participants per alliance
JV_data_dist['emp_min'] = fooReal.min(axis=1)

### Count public companies per alliance

In [16]:
pattern = r'\\n'

JV_data_dist['public_count'] = (JV_data_dist['ppubc'].str.split(pattern, expand=True) == 'P').sum(axis=1)

In [17]:
#Export
JV_data_dist.to_csv("./data/JV_data_dist.csv")

In [18]:
pd.options.display.max_columns = None
JV_data_dist['SNATION_PARTAL'.lower()].value_counts()

N    775
Y    214
Name: snation_partal, dtype: int64